In [4]:
import pandas as pd
import numpy as np
import os
from preprocess import load_data, create_features

In [7]:
def load_data(data_path):
    """Load the dataframes from the preprocessed files (non statistical engineering)

    Arguments:
        data_path {str} -- data folder
    """
    incidents = pd.read_csv(os.path.join(data_path, "incidents.csv"), index_col=0)
    print(f"Incidents shape: {incidents.shape}")
    reports = pd.read_csv(os.path.join(data_path, "reports.csv"))
    print(f"Reports shape: {reports.shape}")
    resources = pd.read_csv(os.path.join(data_path, "resources.csv"), index_col=0)
    print(f"Resources shape: {resources.shape}")
    incidents.merge(reports, left_on="INCIDENT_IDENTIFIER", right_on="INC_IDENTIFIER")
    merged = incidents.merge(
        reports, left_on="INCIDENT_IDENTIFIER", right_on="INC_IDENTIFIER"
    )
    #! Important: mean and sum lead to very different results for pivot table ie not unique
    resources = pd.read_csv(os.path.join(input_path, "resources.csv")).rename(columns={"report_id": "INC209R_IDENTIFIER"})

    result = merged.merge(resources, on="INC209R_IDENTIFIER")
    return result

input_path = "../../data/cleaned"
output_path = "../../data/preprocessed"
target = "diff_incident_area"
features = ["report_number", "incident_area", "cause_id", "month", "year", "STATUS"]
identifiers = ["fire_id", "report_id"]

df = load_data(input_path)

Incidents shape: (94099, 35)
Reports shape: (45389, 23)


C:\Users\benja\AppData\Local\Temp\ipykernel_15796\2701855990.py:9: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  reports = pd.read_csv(os.path.join(data_path, "reports.csv"))


Resources shape: (38568, 141)


In [9]:
df['CY']

0        2015.0
1        2015.0
2        2015.0
3        2015.0
4        2015.0
          ...  
38500    2018.0
38501    2018.0
38502    2018.0
38503    2018.0
38504    2018.0
Name: CY, Length: 38505, dtype: float64

In [ ]:
def create_features(df):
    """Create feature for each day of a fire
    1. Lag/rolling variables, anything that takes into account the past
    2. Target variables
    Arguments:
        df {DataFrame} -- merged dataframe: incidents, reports, resources. A row is a report.
        features {List[str]} -- List of features to use for the task.
        target {str} -- target to predict
    """

    # RENAME COLUMNS
    col_map = {
        "INCIDENT_IDENTIFIER": "fire_id",
        "INC209R_IDENTIFIER": "report_id",
        "year_y": "year",
        "CURR_INCIDENT_AREA": "incident_area",
        "INC209RU_IDENTIFIER": "resource_id",
        "INC209R_IDENTIFIER": "report_id",
        "RESOURCE_QUANTITY": "quantity",
        "RESOURCE_PERSONNEL": "personnel",
        "CURR_INCIDENT_AREA": "area",

    }
    df = df.copy().rename(columns=col_map)
    # Time window select
    df = df[df["year"] > 2013]
    df = df.sort_values(by=["fire_id", "mean_report_date"])
    df['date'] = pd.to_datetime(df['mean_report_date'])
    # Time features
    df["report_number"] = df.groupby("fire_id").cumcount() + 1
    df['prev_area'] = df.groupby('fire_id').shift(1)['area']
    df['next_area'] = df.groupby('fire_id').shift(-1)['area']
    df['prev_date'] = df.groupby('fire_id').shift(1)['date']
    df['next_date'] = df.groupby('fire_id').shift(-1)['date']
    df['prev_date_diff'] = (df['date'] - df['prev_date']).dt.total_seconds()/(24*3600)
    df['next_date_diff'] = (df['next_date'] - df['date']).dt.total_seconds()/(24*3600)
    df['prev_area_diff'] = df['area'] - df['prev_area']
    df['next_area_diff'] = df['next_area'] - df['area']
    df['prev_derivate'] = df['prev_area_diff'] / df['prev_date_diff']
    df['next_derivate'] = df['next_area_diff'] / df['next_date_diff']
    df["will_grow"] = df["next_area_diff"] > 0
    df['time_to_first_report'] = (df['date'] - df.groupby('fire_id')['date'].transform('min')).dt.total_seconds()/(24*3600)
    return df



In [6]:

print(f"Initial shape: {df.shape}")
full_df = create_features(df)

NameError: name 'df' is not defined

In [11]:
full_df.columns

Index(['ID', 'fire_id', 'INCIDENT_NUMBER', 'DONWCGU_PROT_UNIT_IDENTIFIER',
       'INCIDENT_NAME', 'CAUSE_IDENTIFIER', 'DISCOVERY_DATE', 'INCIDENT_AREA',
       'INC_AREA_UOM_IDENTIFIER', 'EST_IM_COST_TO_DATE',
       ...
       'prev_date', 'next_date', 'prev_date_diff', 'next_date_diff',
       'prev_area_diff', 'next_area_diff', 'prev_derivate', 'next_derivate',
       'will_grow', 'time_to_first_report'],
      dtype='object', length=209)

In [12]:
full_df['time_to_first_report'].group

283      0.000000
282      1.593750
284      2.593750
285      4.114583
287      0.000000
           ...   
37379    0.000000
37375    1.203125
37376    1.947917
37378    2.864583
37377    3.864583
Name: time_to_first_report, Length: 38505, dtype: float64